<a href="https://colab.research.google.com/github/GuiXu40/deeplearning0/blob/main/Basic_code/5%E7%A7%8Dnormlization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn

In [7]:
batch_size = 2
time_step = 3
embedding_dim = 4
inputx = torch.rand(batch_size, time_step, embedding_dim)
print(inputx)

tensor([[[0.9074, 0.7389, 0.1134, 0.2540],
         [0.1806, 0.7422, 0.6703, 0.6680],
         [0.7230, 0.1485, 0.3611, 0.1365]],

        [[0.1661, 0.8307, 0.4534, 0.3597],
         [0.6753, 0.0251, 0.8871, 0.5333],
         [0.6172, 0.2389, 0.6213, 0.9127]]])


In [13]:
# 1. batch_nrom 验证API
# NLP: [N, L, C] -> [C]
#CV: [N, C, H, W] -> [C]
## 调用 batch_norm API 
batch_norm_op = torch.nn.BatchNorm1d(embedding_dim, affine=False)
bn_y = batch_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)

## 手写 batch_norm
bn_mean2 = inputx.mean(dim=(0, 1)) #(3x4)

print(bn_mean2)


bn_mean = inputx.mean(dim=(0, 1), keepdim=True)
bn_std = inputx.std(dim=(0, 1), unbiased=False, keepdim=True)
verify_bn_y = (inputx - bn_mean) / (bn_std+1e-5)

print(bn_y)
print(bn_y.shape)
print(verify_bn_y)

tensor([0.5449, 0.4541, 0.5178, 0.4774])
tensor([[[ 1.3067,  0.8793, -1.6447, -0.8555],
         [-1.3136,  0.8894,  0.6204,  0.7301],
         [ 0.6421, -0.9431, -0.6374, -1.3056]],

        [[-1.3657,  1.1626, -0.2617, -0.4508],
         [ 0.4701, -1.3240,  1.5024,  0.2143],
         [ 0.2605, -0.6642,  0.4210,  1.6675]]])
torch.Size([2, 3, 4])
tensor([[[ 1.3068,  0.8793, -1.6448, -0.8555],
         [-1.3136,  0.8894,  0.6204,  0.7301],
         [ 0.6421, -0.9431, -0.6375, -1.3056]],

        [[-1.3657,  1.1626, -0.2617, -0.4508],
         [ 0.4701, -1.3240,  1.5024,  0.2143],
         [ 0.2605, -0.6642,  0.4211,  1.6675]]])


In [15]:
# 2. 实现 layer_norm 并验证API
# NLP: [N, L, C] -> [N, L]
#CV: [N, C, H, W] -> [N, H, W]
layer_norm_op = torch.nn.LayerNorm(embedding_dim, elementwise_affine=False)
ln_y = layer_norm_op(inputx)
ln_mean = inputx.mean(dim=-1, keepdim=True)
ln_std = inputx.std(dim=-1, keepdim=True, unbiased=False)
verify_ln_y = (inputx - ln_mean) / (ln_std+1e-5)
print(ln_y)
print(verify_ln_y)

tensor([[[ 1.2277,  0.7158, -1.1854, -0.7581],
         [-1.7164,  0.7896,  0.4686,  0.4583],
         [ 1.6045, -0.8165,  0.0791, -0.8672]],

        [[-1.1847,  1.5647,  0.0039, -0.3840],
         [ 0.4568, -1.5901,  1.1235,  0.0098],
         [ 0.0821, -1.4990,  0.0993,  1.3175]]])
tensor([[[ 1.2277,  0.7158, -1.1854, -0.7582],
         [-1.7165,  0.7896,  0.4686,  0.4584],
         [ 1.6046, -0.8165,  0.0792, -0.8672]],

        [[-1.1847,  1.5648,  0.0039, -0.3840],
         [ 0.4568, -1.5902,  1.1236,  0.0098],
         [ 0.0821, -1.4990,  0.0993,  1.3176]]])


In [17]:
# 3. 实现 Instance_norm 并验证API
# NLP: [N, L, C] -> [N, C]
#CV: [N, C, H, W] -> [N, C]
Ins_norm_op = torch.nn.InstanceNorm1d(embedding_dim)
Ins_y = Ins_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)
Ins_mean = inputx.mean(dim=1, keepdim=True)
Ins_std = inputx.std(dim=1, keepdim=True, unbiased=False)
verify_Ins_y = (inputx - Ins_mean) / (Ins_std+1e-5)
print(Ins_y)
print(verify_Ins_y)

tensor([[[ 0.9845,  0.7012, -1.1771, -0.4335],
         [-1.3714,  0.7129,  1.2672,  1.3824],
         [ 0.3869, -1.4141, -0.0901, -0.9489]],

        [[-1.4064,  1.3670, -1.1228, -1.0489],
         [ 0.8309, -0.9971,  1.3057, -0.2969],
         [ 0.5754, -0.3698, -0.1829,  1.3458]]])
tensor([[[ 0.9845,  0.7012, -1.1771, -0.4335],
         [-1.3714,  0.7129,  1.2673,  1.3825],
         [ 0.3869, -1.4141, -0.0901, -0.9490]],

        [[-1.4064,  1.3670, -1.1229, -1.0489],
         [ 0.8310, -0.9972,  1.3059, -0.2969],
         [ 0.5755, -0.3699, -0.1829,  1.3459]]])


In [19]:
# 4. gruop Normalization
# NLP: [N,G, L, C//G] -> [N, G]
#CV: [N, G, C//G, H, W] -> [N, G]
num_groups = 2
group_norm_op = torch.nn.GroupNorm(num_groups, embedding_dim, affine=False)
gn_y =  group_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)

group_inputxs = torch.split(inputx, split_size_or_sections=embedding_dim // num_groups, dim=-1)
results = []
for g_inputxs in group_inputxs:
  gn_mean = g_inputxs.mean(dim=(1, 2), keepdim=True)
  gn_std = g_inputxs.std(dim=(1, 2), keepdim=True, unbiased=False)
  gn_result = (g_inputxs-gn_mean) / (gn_std + 1e-5)
  results.append(gn_result)
verify_gn_y = torch.cat(results, dim=-1)
print(gn_y)
print(verify_gn_y)

tensor([[[ 1.1292,  0.5597, -1.1113, -0.4958],
         [-1.3285,  0.5707,  1.3272,  1.3172],
         [ 0.5058, -1.4368, -0.0269, -1.0104]],

        [[-0.8764,  1.3687, -0.8386, -1.2893],
         [ 0.8436, -1.3526,  1.2457, -0.4546],
         [ 0.6472, -0.6306, -0.0320,  1.3688]]])
tensor([[[ 1.1292,  0.5597, -1.1114, -0.4958],
         [-1.3286,  0.5707,  1.3273,  1.3173],
         [ 0.5058, -1.4369, -0.0269, -1.0104]],

        [[-0.8764,  1.3687, -0.8387, -1.2894],
         [ 0.8437, -1.3527,  1.2458, -0.4547],
         [ 0.6473, -0.6306, -0.0320,  1.3689]]])


In [20]:
linear = nn.Linear(embedding_dim, 3, bias=False)
wn_linear = torch.nn.utils.weight_norm(linear)
wn_linear_output = wn_linear(inputx)

weight_direction = linear.weight/(linear.weight.norm(dim=1, keepdim=True))
weight_magnitude = wn_linear.weight_g
verify_wn_linear_output = inputx @ (weight_direction.transpose(-1, -2)) * (weight_magnitude.transpose(-1, -2))
print(wn_linear_output)
print(verify_wn_linear_output)

tensor([[[ 0.4367,  0.2618,  0.5106],
         [ 0.3386, -0.0457,  0.3249],
         [ 0.1599,  0.1467,  0.2196]],

        [[ 0.3504,  0.1031,  0.3616],
         [ 0.1173, -0.0453,  0.1445],
         [ 0.2427, -0.1977,  0.2297]]], grad_fn=<UnsafeViewBackward0>)
tensor([[[ 0.4367,  0.2618,  0.5106],
         [ 0.3386, -0.0457,  0.3249],
         [ 0.1599,  0.1467,  0.2196]],

        [[ 0.3504,  0.1031,  0.3616],
         [ 0.1173, -0.0453,  0.1445],
         [ 0.2427, -0.1977,  0.2297]]], grad_fn=<MulBackward0>)
